In [ ]:
'''
	Table: library_books

	+------------------+---------+
	| Column Name      | Type    |
	+------------------+---------+
	| book_id          | int     |
	| title            | varchar |
	| author           | varchar |
	| genre            | varchar |
	| publication_year | int     |
	| total_copies     | int     |
	+------------------+---------+
	book_id is the unique identifier for this table.
	Each row contains information about a book in the library, including the total number of copies owned by the library.
	Table: borrowing_records

	+---------------+---------+
	| Column Name   | Type    |
	+---------------+---------+
	| record_id     | int     |
	| book_id       | int     |
	| borrower_name | varchar |
	| borrow_date   | date    |
	| return_date   | date    |
	+---------------+---------+
	record_id is the unique identifier for this table.
	Each row represents a borrowing transaction and return_date is NULL if the book is currently borrowed and hasn't been returned yet.

	Write a solution to find all books that are currently borrowed (not returned) and have zero copies available in the library.
	A book is considered currently borrowed if there exists a borrowing record with a NULL return_date
	Return the result table ordered by current borrowers in descending order, then by book title in ascending order.

	The result format is in the following example.

	Example:

	Input:

	library_books table:

	+---------+------------------------+------------------+----------+------------------+--------------+
	| book_id | title                  | author           | genre    | publication_year | total_copies |
	+---------+------------------------+------------------+----------+------------------+--------------+
	| 1       | The Great Gatsby       | F. Scott         | Fiction  | 1925             | 3            |
	| 2       | To Kill a Mockingbird  | Harper Lee       | Fiction  | 1960             | 3            |
	| 3       | 1984                   | George Orwell    | Dystopian| 1949             | 1            |
	| 4       | Pride and Prejudice    | Jane Austen      | Romance  | 1813             | 2            |
	| 5       | The Catcher in the Rye | J.D. Salinger    | Fiction  | 1951             | 1            |
	| 6       | Brave New World        | Aldous Huxley    | Dystopian| 1932             | 4            |
	+---------+------------------------+------------------+----------+------------------+--------------+
	borrowing_records table:

	+-----------+---------+---------------+-------------+-------------+
	| record_id | book_id | borrower_name | borrow_date | return_date |
	+-----------+---------+---------------+-------------+-------------+
	| 1         | 1       | Alice Smith   | 2024-01-15  | NULL        |
	| 2         | 1       | Bob Johnson   | 2024-01-20  | NULL        |
	| 3         | 2       | Carol White   | 2024-01-10  | 2024-01-25  |
	| 4         | 3       | David Brown   | 2024-02-01  | NULL        |
	| 5         | 4       | Emma Wilson   | 2024-01-05  | NULL        |
	| 6         | 5       | Frank Davis   | 2024-01-18  | 2024-02-10  |
	| 7         | 1       | Grace Miller  | 2024-02-05  | NULL        |
	| 8         | 6       | Henry Taylor  | 2024-01-12  | NULL        |
	| 9         | 2       | Ivan Clark    | 2024-02-12  | NULL        |
	| 10        | 2       | Jane Adams    | 2024-02-15  | NULL        |
	+-----------+---------+---------------+-------------+-------------+
	Output:

	+---------+------------------+---------------+-----------+------------------+-------------------+
	| book_id | title            | author        | genre     | publication_year | current_borrowers |
	+---------+------------------+---------------+-----------+------------------+-------------------+
	| 1       | The Great Gatsby | F. Scott      | Fiction   | 1925             | 3                 |
	| 3       | 1984             | George Orwell | Dystopian | 1949             | 1                 |
	+---------+------------------+---------------+-----------+------------------+-------------------+
	Explanation:

	The Great Gatsby (book_id = 1):
	Total copies: 3
	Currently borrowed by Alice Smith, Bob Johnson, and Grace Miller (3 borrowers)
	Available copies: 3 - 3 = 0
	Included because available_copies = 0
	1984 (book_id = 3):
	Total copies: 1
	Currently borrowed by David Brown (1 borrower)
	Available copies: 1 - 1 = 0
	Included because available_copies = 0
	Books not included:
	To Kill a Mockingbird (book_id = 2): Total copies = 3, current borrowers = 2, available = 1
	Pride and Prejudice (book_id = 4): Total copies = 2, current borrowers = 1, available = 1
	The Catcher in the Rye (book_id = 5): Total copies = 1, current borrowers = 0, available = 1
	Brave New World (book_id = 6): Total copies = 4, current borrowers = 1, available = 3
	Result ordering:
	The Great Gatsby appears first with 3 current borrowers
	1984 appears second with 1 current borrower
	Output table is ordered by current_borrowers in descending order, then by book_title in ascending order.
'''

In [25]:
import pandas as pd
def find_books_with_no_available_copies(library_books: pd.DataFrame, borrowing_records: pd.DataFrame) -> pd.DataFrame:
    # Filter books that are currently borrowed (Remove null records)
    borrowing_records=borrowing_records[borrowing_records["return_date"].isna()]
    # calculate the number of book
    borrowing_records=borrowing_records.groupby("book_id").agg(
        current_borrowers=('book_id','count')
    ).reset_index()

    # joining the two dataframe
    df=library_books.merge(borrowing_records,on="book_id")
    df["remaining"]=df["total_copies"]-df["current_borrowers"]
    df=(df[df["remaining"]==0]
            .drop(columns=["remaining","total_copies"])
            .sort_values(by=["current_borrowers","title"], ascending=[False,True])
        )
    return df    

In [27]:
# Sample Library Books Data
library_books_data = {
    "book_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 
                11, 12, 13, 14, 15, 16, 17, 18],
    "title": ["Becoming", "The Catcher in the Rye", "Educated", "Thinking Fast and Slow",
              "Harry Potter", "Great Expectations", "The Chronicles of Narnia", "Outlander",
              "The Time Traveler's Wife", "Wuthering Heights", "The Lord of the Rings",
              "The Fault in Our Stars", "To Kill a Mockingbird", "The Great Gatsby",
              "Me Before You", "The Immortal Life", "The Notebook", "Sapiens"],
    "author": ["Michelle Obama", "J.D. Salinger", "Tara Westover", "Daniel Kahneman",
               "J.K. Rowling", "Charles Dickens", "C.S. Lewis", "Diana Gabaldon",
               "Audrey Niffenegger", "Emily Bronte", "J.R.R. Tolkien", "John Green",
               "Harper Lee", "F. Scott Fitzgerald", "Jojo Moyes", "Rebecca Skloot",
               "Nicholas Sparks", "Yuval Noah Harari"],
    "genre": ["Non-Fiction", "Fiction", "Non-Fiction", "Non-Fiction", "Fantasy",
              "Fiction", "Fantasy", "Romance", "Romance", "Fiction", "Fantasy",
              "Romance", "Fiction", "Fiction", "Romance", "Non-Fiction", "Romance", "Non-Fiction"],
    "publication_year": [1994, 1904, 1964, 1983, 1957, 2003, 1935, 1911, 1977, 1958,
                         1970, 1924, 1998, 2010, 1981, 1945, 1977, 1959],
    "total_copies": [2, 1, 3, 4, 3, 2, 2, 2, 2, 3, 2, 4, 2, 2, 4, 2, 3, 2]
}

borrowing_records_data = {
    "record_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                  11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
                  21, 22, 23, 24, 25, 26, 27, 28, 29],
    "book_id": [4, 4, 5, 5, 5, 6, 8, 8, 10, 10,
                10, 11, 11, 11, 12, 12, 12, 12, 13, 13,
                14, 15, 16, 17, 17, 18, 18, 18, 18],
    "borrower_name": ["Mia Anderson", "Wendy Jackson", "Emma Anderson", "Ruby Anderson", 
                      "Mia White", "Quinn Garcia", "Uma Wilson", "Victor Martin",
                      "Mia Rodriguez", "Alice Lee", "Yolanda Anderson", "Noah Wilson",
                      "Alice Hall", "Xavier Anderson", "David Lewis", "Quinn Rodriguez",
                      "Ruby Martinez", "Paul Smith", "Henry Johnson", "Carol Brown",
                      "Emma Lee", "Ivy Martinez", "Wendy Thomas", "Olivia Martinez",
                      "Carol Jackson", "Henry Smith", "Wendy Lewis", "Henry Brown",
                      "Carol Martinez"],
    "borrow_date": [
        "2024-01-05", "2024-01-14", "2024-01-09", "2024-02-17", "2024-01-15",
        "2024-01-06", "2024-02-20", "2024-02-08", "2024-01-30", "2024-02-16",
        "2024-02-19", "2024-01-30", "2024-02-26", "2024-02-02", "2024-01-20",
        "2024-01-13", "2024-02-28", "2024-01-08", "2024-01-16", "2024-02-16",
        "2024-01-31", "2024-02-25", "2024-01-26", "2024-01-29", "2024-01-02",
        "2024-01-05", "2024-01-04", "2024-02-27", "2024-01-16"
    ],
    "return_date": [
        "2024-01-30", "2024-02-02", "2024-02-02", "2024-03-13", None,
        None, None, None, "2024-02-23", "2024-03-11",
        "2024-02-29", None, None, None, None, "2024-01-25",
        "2024-03-16", None, None, "2024-02-25", None,"2024-03-09",
        None, "2024-02-12", "2024-01-16", None, None,
        "2024-03-15", "2024-02-07"
    ]
}

# Create DataFrames
library_books = pd.DataFrame(library_books_data)
borrowing_records = pd.DataFrame(borrowing_records_data)


# Invoke function
find_books_with_no_available_copies(library_books,borrowing_records)



,book_id,title,author,genre,publication_year,current_borrowers
2,8,Outlander,Diana Gabaldon,Romance,1911,2
8,18,Sapiens,Yuval Noah Harari,Non-Fiction,1959,2
